In [1]:
import pandas as pd 
import numpy as np
import datetime
import calendar
import alphalens
import heapq
import matplotlib.dates as dt
import matplotlib.pyplot as plt
import time, datetime, calendar

# 设置matplotlib图像风格
#plt.style.use('fivethirtyeight')
#plt.rcParams['font.sans-serif']=['SimHei']

In [ ]:
class BT:
    """
    输入：时间、历史数据、调仓（表格从上到下，时间从近到远）
    """
    

    def __init__(self,df_date,df_history,df_transfer,initial_asset=100):
        self.df_date=df_date
        self.initial_asset=initial_asset
        self.rf=0
        self.df_history=df_history
        self.df_transfer=df_transfer
        df1=pd.DataFrame({'asset':(self.df_history*self.df_transfer).sum(axis=1)
            ,'bool':(abs(self.df_transfer-self.df_transfer.shift(-1)).sum(axis=1)).apply(lambda x:x>0)})
        df2=((self.df_history*self.df_transfer).sum(axis=1)/((self.df_history*self.df_transfer.shift(-1)).fillna(1/self.df_history.shape[1]).sum(axis=1)))[df1['bool']]
        df2=((df2[::-1]).cumprod())[::-1]
        df2=pd.DataFrame(df2,columns=['times'])
        df1=pd.concat([df1,df2],axis=1)
        df1.iloc[-1,-1]=1
        df1['times']=df1['times'].fillna(method='bfill')
        df_asset=df1['asset']/(df1['times']*df1['asset'].iloc[-1])*self.initial_asset
        df_return=(df_asset-df_asset.shift(-1))/df_asset.shift(-1)
        self.df_asset=df_asset
        self.df_return = df_return
    
    
    def set_rf(self,rf):
        """
        设定无风险收益率
        """
        self.rf=rf
        

    
    def info(self):
        print("年化收益率：%.4f"%(self.annualized_return_rate())+"\n")
        print("年化波动率：%.4f"%(self.annualized_volatiliy())+"\n")
        print("夏普比率：%.4f"%((self.sharpe()))+"\n")
        print("最大回测：%.4f"%(self.max_drawdown())+"\n")
        print("胜率：%.4f"%(self.win())+"\n")
        return [self.annualized_return_rate(),self.annualized_volatiliy(),self.annualized_return_rate()/self.annualized_volatiliy(),self.max_drawdown(),self.win()]
        
    #回测指标
    
    #年化收益率
    def annualized_return_rate(self):
        df_asset=self.df_asset
        df_asset=np.hstack([df_asset,[100,100,100]])
        YR=((df_asset[::12]-df_asset[::-12][::-1])/df_asset[::12][::-1]).mean()
        return YR
        
    #年化波动率
    def annualized_volatiliy(self):
        df = self.df_return.dropna()
        YV = np.std(df)*np.sqrt(12)
        return YV
    
    #最大回测
    def max_drawdown(self):
        df = self.df_asset.dropna()
        md = ((df.cummin()-df)/df).min()
        return md
    
    #胜率
    def win(self):
        df = self.df_return.dropna()
        df_bool = df.apply(lambda x:x>0)
        w=df_bool.sum()/df.shape[0]
        return w
    
    #夏普比率
    def sharpe(self):
        df = self.df_return.dropna()
        sp = ((df.sum())/(df.shape[0]))/np.std(df)*np.sqrt(12)
        return sp
            